<a href="https://colab.research.google.com/github/PosgradoMNA/actividades-del-projecto-team-1/blob/main/Reto/Reto_Equipo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Maestría en Inteligencia Artificial Aplicada
##Curso: Ciencia y analítica de datos
Tecnológico de Monterrey

Semana 4 - Actividad 1. PCA

Prof Maria de la Paz Rico

Equipo 1

Alumno: Joel Orlando Hernández Ramos

Matrícula: A00759664

Alumno: Rafael Sánchez García

Matrícula: A01182780

Fecha de entrega: Noviembre 16th, 2022
